In [89]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df = sqlContext.read.json("s3n://patricks3db/reviews_Movies_and_TV_small.json")
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [90]:
# check mrubash1/Origin for downloading and updating to s3

In [165]:
ratings = df
ratings = ratings.drop("helpful")
#ratings = ratings.drop("reviewText")
ratings = ratings.drop("reviewTime")
#ratings = ratings.drop("reviewerName")
ratings = ratings.drop("summary")
#ratings = ratings.drop("unixReviewTime")
ratings.columns

['asin',
 'overall',
 'reviewText',
 'reviewerID',
 'reviewerName',
 'unixReviewTime']

In [393]:
from nltk.corpus import wordnet as wn
import re
moodList = wn.synsets('fear')
finalMoodList = [list(mood.closure(lambda s: s.hyponyms())) for mood in moodList]
finalMoodList = [item.lemma_names() for moodList in finalMoodList for item in moodList]
finalMoodList = [item for moodList in finalMoodList for item in moodList]
pattern = re.compile(r'[A-Z]|\_|\-')
finalMoodList = [elem for elem in finalMoodList if not pattern.findall(elem)]
#finalMoodList = [list(mood.closure(lambda s: s.hypernyms())) for mood in moodList]
finalMoodList = set(finalMoodList)
finalMoodList = sorted(finalMoodList)
finalMoodList
#hypo = lambda s: s.hyponyms()
#hyper = lambda s: s.hypernyms()
#list(dog.closure(hypo, depth=1)) == dog.hyponyms()
#list(dog.closure(hyper, depth=1)) == dog.hypernyms()
#list(dog.closure(hypo))

[u'affright',
 u'alarm',
 u'apprehension',
 u'apprehensiveness',
 u'boding',
 u'chill',
 u'consternation',
 u'creeps',
 u'diffidence',
 u'dismay',
 u'dread',
 u'enshrine',
 u'foreboding',
 u'freak',
 u'frisson',
 u'gloom',
 u'gloominess',
 u'hesitance',
 u'hesitancy',
 u'horror',
 u'hysteria',
 u'intimidation',
 u'pall',
 u'panic',
 u'premonition',
 u'presage',
 u'presentiment',
 u'quiver',
 u'saint',
 u'scare',
 u'shadow',
 u'shiver',
 u'shudder',
 u'shyness',
 u'somberness',
 u'sombreness',
 u'suspense',
 u'swivet',
 u'terror',
 u'thrill',
 u'timidity',
 u'timidness',
 u'timorousness',
 u'tingle',
 u'trepidation',
 u'unassertiveness',
 u'worship']

In [388]:
wn.morphy('plays')

u'play'

In [389]:
testStr = "I bought this for my husband who plays the piano.  He is having a wonderful time playing these old hymns.  The music  is at times hard to read because we think the book was published for singing from more than playing from.  Great purchase though!"

In [390]:
len(testStr)

247

In [391]:
from collections import Counter
import nltk
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
cnt = Counter()
for word in tknzr.tokenize(testStr):
    word = wn.morphy(word)
    if word in finalMoodList: cnt[word] += 1
    #cnt[word] += 1
cnt


Counter()

In [ ]:
reviewerSummaryRDD1 = rating.map(lambda row: (row.asin, row.reviewText))

In [259]:
from nltk.corpus import movie_reviews
import random
documents = [(list(movie_reviews.words(fileid)), category) \
    for category in movie_reviews.categories() \
    for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features
#print(document_features(movie_reviews.words('pos/cv957_8737.txt'))) 
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)
classifier.show_most_informative_features(5)

Most Informative Features
          contains(sans) = True              neg : pos    =      8.2 : 1.0
    contains(mediocrity) = True              neg : pos    =      7.5 : 1.0
     contains(dismissed) = True              pos : neg    =      7.1 : 1.0
   contains(overwhelmed) = True              pos : neg    =      6.5 : 1.0
     contains(uplifting) = True              pos : neg    =      6.3 : 1.0


In [166]:
#numRatings = ratings.count()
#numRatings

In [167]:
#users = ratings.map(lambda r: r.reviewerID).distinct().zipWithIndex()
#users.take(5)

In [168]:
#numUsers = users.count()

In [169]:
#numUsers

In [170]:
#numUsersPartitions = users.getNumPartitions()
#userids = sc.parallelize(range(numUsers), numUsersPartitions)
#usersWithID = users.zip(userids)

In [171]:
#users = ratings.map(lambda r: r.reviewerID).distinct().zipWithIndex()

In [172]:
#movies = ratings.map(lambda r: r.asin).distinct().zipWithIndex()
#movies = ratings.map(lambda r: r.asin).distinct().collect()
#movies = dict([(y,x) for x,y in enumerate(movies)])

In [173]:
#moviesDf = sqlContext.createDataFrame(movies, ['asin', 'movieID'])
#moviesDf.take(5)
#moviesDf = sqlContext.createDataFrame(movies)
#type(movies)

In [174]:
#usersDf = sqlContext.createDataFrame(users, ['reviewerID', 'userID'])
#usersDf.take(5)

In [175]:
#rawRatings1 = ratings.join(usersDf, ratings.reviewerID == usersDf.reviewerID).drop("reviewerID")
#rawRatings1.printSchema()


In [177]:
#rawRatings2 = rawRatings1.join(moviesDf, ratings.asin == moviesDf.asin).drop("asin")
#rawRatings2.printSchema()

In [111]:
#from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [112]:
#import uuid
#testAsin = 'A1KBT3D1O0LA7L'
#testAsinInt = int(id('A1KBT3D1O0LA7L'))
#testAsinInt

In [113]:
#rawRatings3 = rawRatings2.map(lambda row: (row.userID, row.movieID, row.overall))
#ratings = ratings.map(lambda row: (users[row.reviewerID], movies[row.asin], row.overall))

In [178]:
#rawRatings3.take(10)

In [115]:
#rank = 10
#numIterations = 10
#model = ALS.train(rawRatings3, rank, numIterations)

In [179]:
#type(model)

In [180]:
#type(ALS)

In [181]:
#userMoviePair = usersDf.join(moviesDf).drop('reviewerID').drop('asin').map(lambda row: (row.userID, row.movieID))

In [182]:
#userMoviePair.take(5)
#type(userMoviePair)

In [183]:
#recommendations = model.predictAll(userMoviePair)

In [ ]:
#type(recommendations)

In [ ]:
#recommendations.take(5)